In [1]:
import pandas as pd
import numpy as np
from glob import glob
import pickle
from collections import Counter
import os

In [2]:
session_lst = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20']
script_lst = ['script01','script01','script01','script01','script01','script01','impro01','impro02','impro03','impro04']
sex_lst = ['F', 'M']

In [3]:
import pandas as pd

sid_dict_F = {}
sid_lst_F = []

for session in session_lst:
    df = pd.read_csv(f'/home/arplab/project/paradeigma/multi_modal/KEMDy19/annotation/Session{session}_F_res.csv', skiprows=1)
    df.rename(columns = {'Unnamed: 9' : 'Segment ID'}, inplace = True)
    sid_dict_F[session] = df['Segment ID']
    sid_lst_F.extend(list(df['Segment ID']))
    
len(sid_lst_F)

10283

In [5]:
sid_dict_M = {}
sid_lst_M = []
for session in session_lst:
    df = pd.read_csv(f'/home/arplab/project/paradeigma/multi_modal/KEMDy19/annotation/Session{session}_M_res.csv', skiprows=1)
    df.rename(columns = {'Unnamed: 9' : 'Segment ID'}, inplace = True)
    sid_dict_M[session] = df['Segment ID']
    sid_lst_M.extend(list(df['Segment ID']))
    
len(sid_lst_M)

10283

In [6]:
from collections import Counter

sid_counter_m = Counter(sid_lst_M)
[k for k in sid_counter_m if sid_counter_m.get(k) > 1]

[]

In [7]:
from collections import Counter

sid_counter_f = Counter(sid_lst_F)
[k for k in sid_counter_f if sid_counter_f.get(k) > 1]

['Sess03_impro02_M001', 'Sess09_script06_M001', 'Sess09_impro01_M001']

In [8]:
sid_counter_f['Sess03_impro02_M001'],sid_counter_f['Sess09_script06_M001'],sid_counter_f['Sess09_impro01_M001'] 

(2, 2, 2)

In [9]:
set(sid_lst_F) - set(sid_lst_M)

{'Sess04_impro03_M031'}

In [10]:
set(sid_lst_M) - set(sid_lst_F)

{'Sess03_impro02_F001',
 'Sess04_impro03_F031',
 'Sess09_impro01_F001',
 'Sess09_script06_F001'}

소결

1. 중복치

    1) 남자 기준 anno엔 중복치가 없다.
    
    2) 여자 기준 anno엔 'Sess03_impro02_M001', 'Sess09_script06_M001', 'Sess09_impro01_M001'가 중복되어 있다. 
    
2. 결측치

    1) 남자에만 존재하는 sid는 다음과 같다.
        :{'Sess03_impro02_F001','Sess04_impro03_F031', 'Sess09_impro01_F001', 'Sess09_script06_F001 }
        
    2) 여자에만 존재하는 sid는 다음과 같다.
        :{'Sess04_impro03_M031'}

3. 특이사항
    1) {'Sess04_impro03_M031'}, {'Sess04_impro03_F031'}는 둘 다 wav, txt를 갖지 않은 sid다.

4. 최종 전략

    1) 중복치가 발생하지 않은 남자 쪽 anno를 기준으로 sid_lst 골자를 짠다.
    2) 남자 쪽 anno에서 'Sess04_impro03_F031'를 삭제한다.
    3) 'Sess03_impro02_F001', 'Sess09_impro01_F001', 'Sess09_script06_F001를 발화한 여자의 anno는 여자 쪽에서 중복된 것 중 가져와야 한다.
    

In [11]:
sid_lst_M.remove('Sess04_impro03_F031')

In [12]:
sid_19_lst = sid_lst_M

In [13]:
import pandas as pd
from tqdm import tqdm

anno_19_df = pd.DataFrame()
missing_sid = {}
for i, sid in enumerate(tqdm(sid_19_lst)):
    
    session = sid.split('_')[0][-2:]
    sex = sid.split('_')[2][0]
    # session 9에서 script 6번에서 여자가 첫번째로 말한 여자의 eda는 존재하지 않는다. 
    # 따라서, 아예 배제하기 보단 여자의 말을 들은 남자의 감정이, 즉 청자의 감정이 발화자와 같을 것이라고 가정한 뒤에 남자의 감정을 타겟으로 가져온다.
    if sid == 'Sess09_script06_F001':
        anno =  pd.read_csv(f'/home/arplab/project/paradeigma/multi_modal/KEMDy19/annotation/Session{session}_M_res.csv', skiprows=1)
        
    else: 
        anno = pd.read_csv(f'/home/arplab/project/paradeigma/multi_modal/KEMDy19/annotation/Session{session}_{sex}_res.csv', skiprows=1)
        
    
    anno.rename(columns = {'Unnamed: 9' : 'Segment ID'}, inplace = True)
    anno = anno[anno.columns[9:]]
    
    # session 3에서 두 번째 즉흥적으로 한 여자의 첫 번째 발화는 남자쪽 eda에 두 번 기록이 되어있다. 그 두번 기록된 것 중 첫 번째 가 여자의 발화이다. 
    if sid == 'Sess03_impro02_F001' :
        sid = sid.replace('F','M')
        rows = anno[anno['Segment ID'] == sid]
        row = rows.iloc[0]
        row['Segment ID'] = 'Sess03_impro02_F001'
        
    if sid == 'Sess09_impro01_F001':
        sid = sid.replace('F','M')
        rows = anno[anno['Segment ID'] == sid]
        row = rows.iloc[0]
        row['Segment ID'] = 'Sess09_impro01_F001'
        
    else: 
        row = anno[anno['Segment ID'] == sid]
    
    if len(row) == 0:
        missing_sid[i] = sid
    anno_19_df = anno_19_df.append(row, ignore_index=True)

anno_19_df.reset_index(drop=True, inplace = True)

  0%|          | 0/10282 [00:00<?, ?it/s]/tmp/ipykernel_2842002/3633300193.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  anno_19_df = anno_19_df.append(row, ignore_index=True)
/tmp/ipykernel_2842002/3633300193.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  anno_19_df = anno_19_df.append(row, ignore_index=True)
/tmp/ipykernel_2842002/3633300193.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  anno_19_df = anno_19_df.append(row, ignore_index=True)
/tmp/ipykernel_2842002/3633300193.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  anno_19_df = anno_19_df.append(row, ignore_index=True)
/tmp/ipykernel_2842002/3633300193.p

In [14]:
temp = anno_19_df['Segment ID'].copy()
temp[1362] = 'Sess03_impro02_F001'
anno_19_df['Segment ID'] = temp

In [15]:
anno_19_df.drop(1361, axis=0, inplace=True)
anno_19_df.reset_index(drop=True, inplace=True)

In [16]:
emotion_eval_df = anno_19_df[['Emotion.1','Emotion.2','Emotion.3','Emotion.4','Emotion.5','Emotion.6','Emotion.7','Emotion.8','Emotion.9','Emotion.10']]
valence_eval_df = anno_19_df[['Valence.1','Valence.2','Valence.3','Valence.4','Valence.5','Valence.6','Valence.7','Valence.8','Valence.9','Valence.10']]
arousal_eval_df = anno_19_df[['Arousal.1','Arousal.2','Arousal.3','Arousal.4','Arousal.5','Arousal.6','Arousal.7','Arousal.8','Arousal.9','Arousal.10']]

In [17]:
from collections import Counter

annot_target = []
for i in range(len(emotion_eval_df)):
    score_table = {'neutral': 0, 'happy': 0, 'surprise':0, 'disgust': 0, 'angry': 0, 'sad':0, 'fear': 0}
    i_of_tsdata = Counter(emotion_eval_df.iloc[i])
    for emotion in i_of_tsdata.keys():
        score_table[emotion] += i_of_tsdata[emotion]
    annot_target.append(list(score_table.values()))
annot_target[:3]

[[0, 0, 9, 0, 0, 0, 1], [1, 0, 2, 0, 0, 0, 7], [0, 0, 0, 0, 10, 0, 0]]

In [18]:
val_target = []
for i in range(len(valence_eval_df)):
    score_table = {1: 0, 2: 0, 3:0, 4: 0, 5: 0}
    i_of_tsdata = Counter(valence_eval_df.iloc[i])
    for val in i_of_tsdata.keys():
        score_table[val] += i_of_tsdata[val]
    val_target.append(list(score_table.values()))
val_target[:3]

[[4, 5, 1, 0, 0], [5, 5, 0, 0, 0], [7, 3, 0, 0, 0]]

In [19]:
arou_target = []
for i in range(len(arousal_eval_df)):
    score_table = {1: 0, 2: 0, 3:0, 4: 0, 5: 0}
    i_of_tsdata = Counter(arousal_eval_df.iloc[i])
    for arou in i_of_tsdata.keys():
        score_table[arou] += i_of_tsdata[arou]
    arou_target.append(list(score_table.values()))
arou_target[:3]

[[1, 0, 0, 6, 3], [1, 0, 2, 6, 1], [1, 0, 0, 3, 6]]

In [20]:
annotation_19 = anno_19_df.drop(columns = list(anno_19_df.columns[4:]))

In [21]:
annotation_19['emotion_vector'] = annot_target
annotation_19['valence_vector'] = val_target
annotation_19['arousal_vector'] = arou_target

In [24]:
import csv
eda_19_dict = {}
temp_19_dict = {}

for sid in tqdm(sid_19_lst):
    eda_value = []
    temp_value = []
    session = sid.split('_')[0][-2:]
    sex = sid.split('_')[2][0]            
                
    with open(f'/home/arplab/project/paradeigma/multi_modal/KEMDy19/EDA/Session{session}/Original/Sess{session}{sex}.csv') as csvfile:
        eda_csv = csv.reader(csvfile)
        for row in eda_csv:
            if len(row) > 3:
                a, _, _, c = row
                if c == sid:
                    eda_value.append(a)
                    
    with open(f'/home/arplab/project/paradeigma/multi_modal/KEMDy19/TEMP/Session{session}/Original/Sess{session}{sex}.csv') as csvfile:
        temp_csv = csv.reader(csvfile)
        for row in temp_csv:
            if len(row) > 3:
                a, _, _, c = row
                if c == sid:
                    temp_value.append(a)

    eda_19_dict[sid] = eda_value
    temp_19_dict[sid] = temp_value    

100%|██████████| 10282/10282 [06:10<00:00, 27.73it/s]


In [25]:
for i in list(eda_19_dict.keys()):
    if i not in list(temp_19_dict.keys()):
        print(i)
        
for i in list(temp_19_dict.keys()):
    if i not in list(eda_19_dict.keys()):
        print(i)

In [26]:
length = len(eda_19_dict)

eda_19_num = {}
temp_19_num = {}

missing_eda_19_sid = []
missing_temp_19_sid = []

attention_path = []

for num in range(length):
    eda_length = len(list(eda_19_dict.values())[num])
    temp_length = len(list(temp_19_dict.values())[num])
    eda_sid = list(eda_19_dict.keys())[num]
    temp_sid = list(temp_19_dict.keys())[num]
    
    eda_19_num[eda_sid] = eda_length
    temp_19_num[temp_sid] = temp_length
    
    if eda_length == 0:
        missing_eda_19_sid.append(eda_sid)
        
    if temp_length == 0:
        missing_temp_19_sid.append(temp_sid)
                      
    if eda_length != temp_length:
        attention_path.append(eda_sid)                     
                                                

In [27]:
len(missing_eda_19_sid), len(missing_temp_19_sid),attention_path

(1270,
 1274,
 ['Sess01_impro03_F009',
  'Sess04_script06_M019',
  'Sess06_script06_M008',
  'Sess06_script06_M009',
  'Sess06_script06_F017',
  'Sess09_script06_F015'])

In [28]:
eda_19_dict

{'Sess01_script01_M001': ['4.408951',
  '4.403825',
  '4.410233',
  '4.421767',
  '4.429456',
  '4.414751',
  '4.121273',
  '4.222516',
  '4.328886',
  '4.268652',
  '4.399372',
  '4.586480',
  '4.533936',
  '4.328886',
  '4.181506',
  '4.217390',
  '4.349391',
  '4.349391',
  '4.598014',
  '4.286594',
  '4.334012',
  '4.446790',
  '4.580072',
  '4.668500',
  '4.829977',
  '4.923531',
  '4.968386',
  '4.980824',
  '4.965446',
  '4.920591',
  '4.868047',
  '4.793716',
  '4.719386'],
 'Sess01_script01_F001': ['0.145914',
  '0.140794',
  '0.144634',
  '0.145914',
  '0.140794',
  '0.142074',
  '0.143354',
  '0.144634',
  '0.144634',
  '0.143354',
  '0.143354',
  '0.139514',
  '0.138234',
  '0.139514',
  '0.143354',
  '0.143354',
  '0.145914'],
 'Sess01_script01_M002': ['4.478828',
  '4.396809',
  '4.334012',
  '4.322478',
  '4.346828',
  '4.412188',
  '4.444226',
  '4.451916',
  '4.477547',
  '4.548033',
  '4.517275',
  '4.241740',
  '4.195603',
  '4.167409',
  '4.240458',
  '4.445508',
  

In [29]:
ts_19_df = pd.DataFrame( columns=['Segment ID', 'EDA', 'TEMP'])
ts_19_df['Segment ID'] = eda_19_dict.keys()
ts_19_df['EDA'] = eda_19_dict.values()
ts_19_df['TEMP'] = temp_19_dict.values()

In [30]:
annotation_19

,Segment ID,Emotion,Valence,Arousal,emotion_vector,valence_vector,arousal_vector
0,Sess01_script01_M001,surprise,1.7,4.0,"[0, 0, 9, 0, 0, 0, 1]","[4, 5, 1, 0, 0]","[1, 0, 0, 6, 3]"
1,Sess01_script01_F001,fear,1.5,3.6,"[1, 0, 2, 0, 0, 0, 7]","[5, 5, 0, 0, 0]","[1, 0, 2, 6, 1]"
2,Sess01_script01_M002,angry,1.3,4.3,"[0, 0, 0, 0, 10, 0, 0]","[7, 3, 0, 0, 0]","[1, 0, 0, 3, 6]"
3,Sess01_script01_M003,angry,1.5,3.8,"[0, 0, 1, 0, 9, 0, 0]","[6, 3, 1, 0, 0]","[1, 0, 0, 8, 1]"
4,Sess01_script01_F002,fear,1.6,3.7,"[0, 0, 2, 0, 0, 0, 8]","[5, 4, 1, 0, 0]","[1, 0, 1, 7, 1]"
...,...,...,...,...,...,...,...
10277,Sess20_impro04_M018,sad,2.3,2.1,"[3, 0, 0, 0, 0, 7, 0]","[2, 4, 3, 1, 0]","[0, 9, 1, 0, 0]"
10278,Sess20_impro04_F023,sad,2.5,2.3,"[2, 0, 0, 0, 0, 8, 0]","[0, 6, 3, 1, 0]","[1, 5, 4, 0, 0]"
10279,Sess20_impro04_M019,sad,2.1,1.8,"[1, 0, 0, 0, 0, 9, 0]","[4, 2, 3, 1, 0]","[3, 6, 1, 0, 0]"
10280,Sess20_impro04_F024,happy,3.9,3.2,"[3, 7, 0, 0, 0, 0, 0]","[0, 0, 1, 9, 0]","[0, 1, 6, 3, 0]"


In [31]:
annotation_19['EDA'] = ts_19_df['EDA']
annotation_19['TEMP'] = ts_19_df['TEMP']

In [32]:
def make_nan_empty_list(x):
    if type(x) == type(np.NAN):
        return []
    
    else: 
        return x

annotation_19['EDA'] = annotation_19['EDA'].apply(make_nan_empty_list)
annotation_19['TEMP'] = annotation_19['TEMP'].apply(make_nan_empty_list)

In [33]:
annotation_19.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10282 entries, 0 to 10281
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Segment ID      10282 non-null  object 
 1   Emotion         10282 non-null  object 
 2   Valence         10282 non-null  float64
 3   Arousal         10282 non-null  float64
 4   emotion_vector  10282 non-null  object 
 5   valence_vector  10282 non-null  object 
 6   arousal_vector  10282 non-null  object 
 7   EDA             10282 non-null  object 
 8   TEMP            10282 non-null  object 
dtypes: float64(2), object(7)
memory usage: 723.1+ KB


In [34]:
annotation_19['EDA length'] = annotation_19['EDA'].apply(lambda x:len(x))
annotation_19['TEMP length'] = annotation_19['TEMP'].apply(lambda x:len(x))

In [35]:
def from_str_to_list(ts_lst):
    if isinstance(ts_lst, float):
        return ts_lst
    
    result = []
    for ts in ts_lst:
        result.append(float(ts))
        
    return result


annotation_19['EDA'] = annotation_19['EDA'].apply(from_str_to_list)
annotation_19['TEMP'] = annotation_19['TEMP'].apply(from_str_to_list)

In [36]:
annotation_19['Scaled EDA'] = 0
annotation_19['Scaled EDA'] = annotation_19['Scaled EDA'].astype('object')
annotation_19['Scaled TEMP'] = 0
annotation_19['Scaled TEMP'] = annotation_19['Scaled TEMP'].astype('object')

In [37]:
annotation_19.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10282 entries, 0 to 10281
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Segment ID      10282 non-null  object 
 1   Emotion         10282 non-null  object 
 2   Valence         10282 non-null  float64
 3   Arousal         10282 non-null  float64
 4   emotion_vector  10282 non-null  object 
 5   valence_vector  10282 non-null  object 
 6   arousal_vector  10282 non-null  object 
 7   EDA             10282 non-null  object 
 8   TEMP            10282 non-null  object 
 9   EDA length      10282 non-null  int64  
 10  TEMP length     10282 non-null  int64  
 11  Scaled EDA      10282 non-null  object 
 12  Scaled TEMP     10282 non-null  object 
dtypes: float64(2), int64(2), object(9)
memory usage: 1.0+ MB


In [38]:
annotation_19['Emotion'] = annotation_19['Emotion'].apply(lambda x: ';'.join(sorted(x.split(';'))))

In [39]:
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

scaler = StandardScaler()   

session_script_sex = set([x[:-3] for x in sid_19_lst])
length = len(annotation_19)

total_eda_dict = {}
total_temp_dict = {}
for sss in tqdm(session_script_sex):
    
    eda_value = []
    temp_value = []
    for segment in range(length):
        sid = annotation_19['Segment ID'][segment]
        if sss in sid:
            eda_value.extend(annotation_19['EDA'][segment])
            temp_value.extend(annotation_19['TEMP'][segment])
            
    if len(eda_value) != 0:  
        eda_value = scaler.fit_transform(np.array(eda_value).reshape(-1,1))   
        total_eda_dict[sss] = eda_value.reshape(1,-1).tolist()
        
    else: 
        total_eda_dict[sss] = [[]]
        
    if len(temp_value) != 0:  
        temp_value = scaler.fit_transform(np.array(temp_value).reshape(-1,1))   
        total_temp_dict[sss] = temp_value.reshape(1,-1).tolist()    
        
    else: 
        total_temp_dict[sss] = [[]]

100%|██████████| 400/400 [00:31<00:00, 12.58it/s]


In [40]:
for i, segment in enumerate(tqdm(sid_19_lst)):
    
    scaled_sss_eda = total_eda_dict[segment[:-3]][0]
    try : nums = int(annotation_19['EDA length'][annotation_19['Segment ID'] == segment])
    except : nums = 0
    
    scaled_eda = scaled_sss_eda[:nums]
    total_eda_dict[segment[:-3]][0] = scaled_sss_eda[nums:]

    annotation_19['Scaled EDA'][i] = scaled_eda

  0%|          | 0/10282 [00:00<?, ?it/s]/tmp/ipykernel_2842002/1330515395.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  annotation_19['Scaled EDA'][i] = scaled_eda
100%|██████████| 10282/10282 [00:10<00:00, 938.61it/s]


In [41]:
for i, segment in enumerate(tqdm(sid_19_lst)):

    scaled_sss_temp = total_temp_dict[segment[:-3]][0]
    try : nums = int(annotation_19['TEMP length'][annotation_19['Segment ID'] == segment])
    except : nums = 0
        
    scaled_temp = scaled_sss_temp[:nums]
    total_temp_dict[segment[:-3]][0] = scaled_sss_temp[nums:]

    annotation_19['Scaled TEMP'][i] = scaled_temp
    

  0%|          | 0/10282 [00:00<?, ?it/s]/tmp/ipykernel_2842002/3219104382.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  annotation_19['Scaled TEMP'][i] = scaled_temp
100%|██████████| 10282/10282 [00:11<00:00, 932.00it/s]


In [42]:
including_eda_sid = annotation_19['Segment ID'][annotation_19['EDA length'] != 0]
including_temp_sid = annotation_19['Segment ID'][annotation_19['TEMP length'] != 0]

len(including_eda_sid), len(including_temp_sid)

(9012, 9008)

In [43]:
annotation_19

,Segment ID,Emotion,Valence,Arousal,emotion_vector,valence_vector,arousal_vector,EDA,TEMP,EDA length,TEMP length,Scaled EDA,Scaled TEMP
0,Sess01_script01_M001,surprise,1.7,4.0,"[0, 0, 9, 0, 0, 0, 1]","[4, 5, 1, 0, 0]","[1, 0, 0, 6, 3]","[4.408951, 4.403825, 4.410233, 4.421767, 4.429...","[34.66, 34.66, 34.66, 34.66, 34.66, 34.68, 34....",33,33,"[-0.926455937246618, -0.9285481420682529, -0.9...","[1.6900874192514124, 1.6900874192514124, 1.690..."
1,Sess01_script01_F001,fear,1.5,3.6,"[1, 0, 2, 0, 0, 0, 7]","[5, 5, 0, 0, 0]","[1, 0, 2, 6, 1]","[0.145914, 0.140794, 0.144634, 0.145914, 0.140...","[30.69, 30.69, 30.69, 30.69, 30.71, 30.71, 30....",17,17,"[-1.407155706494133, -1.4312942323175069, -1.4...","[-1.1135528407658513, -1.1135528407658513, -1...."
2,Sess01_script01_M002,angry,1.3,4.3,"[0, 0, 0, 0, 10, 0, 0]","[7, 3, 0, 0, 0]","[1, 0, 0, 3, 6]","[4.478828, 4.396809, 4.334012, 4.322478, 4.346...","[34.61, 34.61, 34.61, 34.61, 34.61, 34.61, 34....",27,27,"[-0.8979352590723317, -0.9314117606848334, -0....","[1.6164861115016742, 1.6164861115016742, 1.616..."
3,Sess01_script01_M003,angry,1.5,3.8,"[0, 0, 1, 0, 9, 0, 0]","[6, 3, 1, 0, 0]","[1, 0, 0, 8, 1]","[4.746299, 4.733483, 4.716823, 4.702725, 4.692...","[34.55, 34.55, 34.55, 34.55, 34.55, 34.55, 34....",10,10,"[-0.7887655134914608, -0.7939964337009954, -0....","[1.5281645422019798, 1.5281645422019798, 1.528..."
4,Sess01_script01_F002,fear,1.6,3.7,"[0, 0, 2, 0, 0, 0, 8]","[5, 4, 1, 0, 0]","[1, 0, 1, 7, 1]","[0.134394, 0.135674, 0.134394, 0.134394, 0.133...","[30.69, 30.69, 30.71, 30.71, 30.71, 30.71, 30....",13,13,"[-1.461467389596724, -1.4554327581408808, -1.4...","[-1.1135528407658513, -1.1135528407658513, -0...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10277,Sess20_impro04_M018,sad,2.3,2.1,"[3, 0, 0, 0, 0, 7, 0]","[2, 4, 3, 1, 0]","[0, 9, 1, 0, 0]","[0.550024, 0.547461, 0.54618, 0.543617, 0.5461...","[31.27, 31.27, 31.27, 31.27, 31.27, 31.27, 31....",31,31,"[-1.0596542781342695, -1.091127738705599, -1.1...","[-0.12418082175855716, -0.12418082175855716, -..."
10278,Sess20_impro04_F023,sad,2.5,2.3,"[2, 0, 0, 0, 0, 8, 0]","[0, 6, 3, 1, 0]","[1, 5, 4, 0, 0]","[0.216838, 0.220682, 0.221964, 0.223245, 0.221...","[29.77, 29.77, 29.77, 29.77, 29.77, 29.77, 29....",46,46,"[-0.8593367498709915, -0.7994211382956963, -0....","[-1.3249127202479716, -1.3249127202479716, -1...."
10279,Sess20_impro04_M019,sad,2.1,1.8,"[1, 0, 0, 0, 0, 9, 0]","[4, 2, 3, 1, 0]","[3, 6, 1, 0, 0]","[0.519269, 0.517987, 0.519269, 0.517987, 0.516...","[31.31, 31.31, 31.31, 31.31, 31.29, 31.29, 31....",56,56,"[-1.437323525060235, -1.4530663953108964, -1.4...","[0.697453930425023, 0.697453930425023, 0.69745..."
10280,Sess20_impro04_F024,happy,3.9,3.2,"[3, 7, 0, 0, 0, 0, 0]","[0, 0, 1, 9, 0]","[0, 1, 6, 3, 0]","[0.207868, 0.207868, 0.210431, 0.211712, 0.212...","[29.73, 29.73, 29.73, 29.73, 29.73, 29.73, 29....",52,52,"[-0.9991502347384216, -0.9991502347384216, -0....","[-1.6382728540476073, -1.6382728540476073, -1...."


In [44]:
missing_eda_index = set(annotation_19[annotation_19['EDA length'] == 0].index)
missing_temp_index = set(annotation_19[annotation_19['TEMP length'] == 0].index)

In [45]:
missing_ts_index = missing_eda_index | missing_temp_index
annotation_19_nonmissing = annotation_19.loc[list(set(annotation_19.index) - missing_ts_index)]

In [46]:
annotation_19_nonmissing.reset_index(drop=True, inplace=True)

In [50]:
import pickle

with open('/home/arplab/project/paradeigma/multi_modal/model/data/paradeigma_KEMDY19_annotation_nonmissing.pkl', 'wb') as f:
    pickle.dump(annotation_19_nonmissing, f, pickle.HIGHEST_PROTOCOL)
    

In [1]:
import pickle

with open('/home/arplab/project/paradeigma/multi_modal/model/data/paradeigma_KEMDY19_annotation_nonmissing.pkl', 'rb') as f:
    annotation_19_nonmissing = pickle.load(f)

In [2]:
annotation_19_nonmissing

,Segment ID,Emotion,Valence,Arousal,emotion_vector,valence_vector,arousal_vector,EDA,TEMP,EDA length,TEMP length,Scaled EDA,Scaled TEMP
0,Sess01_script01_M001,surprise,1.7,4.0,"[0, 0, 9, 0, 0, 0, 1]","[4, 5, 1, 0, 0]","[1, 0, 0, 6, 3]","[4.408951, 4.403825, 4.410233, 4.421767, 4.429...","[34.66, 34.66, 34.66, 34.66, 34.66, 34.68, 34....",33,33,"[-0.926455937246618, -0.9285481420682529, -0.9...","[1.6900874192514124, 1.6900874192514124, 1.690..."
1,Sess01_script01_F001,fear,1.5,3.6,"[1, 0, 2, 0, 0, 0, 7]","[5, 5, 0, 0, 0]","[1, 0, 2, 6, 1]","[0.145914, 0.140794, 0.144634, 0.145914, 0.140...","[30.69, 30.69, 30.69, 30.69, 30.71, 30.71, 30....",17,17,"[-1.407155706494133, -1.4312942323175069, -1.4...","[-1.1135528407658513, -1.1135528407658513, -1...."
2,Sess01_script01_M002,angry,1.3,4.3,"[0, 0, 0, 0, 10, 0, 0]","[7, 3, 0, 0, 0]","[1, 0, 0, 3, 6]","[4.478828, 4.396809, 4.334012, 4.322478, 4.346...","[34.61, 34.61, 34.61, 34.61, 34.61, 34.61, 34....",27,27,"[-0.8979352590723317, -0.9314117606848334, -0....","[1.6164861115016742, 1.6164861115016742, 1.616..."
3,Sess01_script01_M003,angry,1.5,3.8,"[0, 0, 1, 0, 9, 0, 0]","[6, 3, 1, 0, 0]","[1, 0, 0, 8, 1]","[4.746299, 4.733483, 4.716823, 4.702725, 4.692...","[34.55, 34.55, 34.55, 34.55, 34.55, 34.55, 34....",10,10,"[-0.7887655134914608, -0.7939964337009954, -0....","[1.5281645422019798, 1.5281645422019798, 1.528..."
4,Sess01_script01_F002,fear,1.6,3.7,"[0, 0, 2, 0, 0, 0, 8]","[5, 4, 1, 0, 0]","[1, 0, 1, 7, 1]","[0.134394, 0.135674, 0.134394, 0.134394, 0.133...","[30.69, 30.69, 30.71, 30.71, 30.71, 30.71, 30....",13,13,"[-1.461467389596724, -1.4554327581408808, -1.4...","[-1.1135528407658513, -1.1135528407658513, -0...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9003,Sess20_impro04_M018,sad,2.3,2.1,"[3, 0, 0, 0, 0, 7, 0]","[2, 4, 3, 1, 0]","[0, 9, 1, 0, 0]","[0.550024, 0.547461, 0.54618, 0.543617, 0.5461...","[31.27, 31.27, 31.27, 31.27, 31.27, 31.27, 31....",31,31,"[-1.0596542781342695, -1.091127738705599, -1.1...","[-0.12418082175855716, -0.12418082175855716, -..."
9004,Sess20_impro04_F023,sad,2.5,2.3,"[2, 0, 0, 0, 0, 8, 0]","[0, 6, 3, 1, 0]","[1, 5, 4, 0, 0]","[0.216838, 0.220682, 0.221964, 0.223245, 0.221...","[29.77, 29.77, 29.77, 29.77, 29.77, 29.77, 29....",46,46,"[-0.8593367498709915, -0.7994211382956963, -0....","[-1.3249127202479716, -1.3249127202479716, -1...."
9005,Sess20_impro04_M019,sad,2.1,1.8,"[1, 0, 0, 0, 0, 9, 0]","[4, 2, 3, 1, 0]","[3, 6, 1, 0, 0]","[0.519269, 0.517987, 0.519269, 0.517987, 0.516...","[31.31, 31.31, 31.31, 31.31, 31.29, 31.29, 31....",56,56,"[-1.437323525060235, -1.4530663953108964, -1.4...","[0.697453930425023, 0.697453930425023, 0.69745..."
9006,Sess20_impro04_F024,happy,3.9,3.2,"[3, 7, 0, 0, 0, 0, 0]","[0, 0, 1, 9, 0]","[0, 1, 6, 3, 0]","[0.207868, 0.207868, 0.210431, 0.211712, 0.212...","[29.73, 29.73, 29.73, 29.73, 29.73, 29.73, 29....",52,52,"[-0.9991502347384216, -0.9991502347384216, -0....","[-1.6382728540476073, -1.6382728540476073, -1...."
